In [15]:
import requests
from groq import Groq

In [13]:
formatted_prompt = "What is the average salary for a software engineer in Zurich in CHF?"

In [14]:
url = "http://dgx-a100.cloudlab.zhaw.ch:9175/generate"
payload = {"inputs": formatted_prompt, "paramters": {"max_new_tokens": 200}}
headers = {"Content-Type": "application/json"}
response = requests.post(url, json=payload, headers=headers)
response.json()

{'generated_text': 'What is the cost of living in Zurich?What are some tips for finding a job as a software engineer in Zurich?What are some tips for relocating to Zurich from another country?What are some things to consider when deciding whether to relocate to Zurich?What are some tips for learning German?What are some fun things to do in Zurich?What are some things to consider when renting a place to live in Zurich?What are some things to consider when buying a place to live in Zurich?What'}

In [17]:
client = Groq(
    api_key="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",
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "What is the average salary of a Engineer in Zurich in CHF? only return the salary range in CHF",
        }
    ],
    model="llama3-70b-8192",
)

print(chat_completion.choices[0].message.content)

According to various sources, the average salary range for an Engineer in Zurich, Switzerland is:

CHF 80,000 - 120,000 per year.
